In [1]:
import pandas as pd

In [2]:
data = pd.read_csv(
    "data/USDCHF_Candlestick_1_Hour_BID_01.02.2021-06.02.2021.csv"
)
data.rename(columns={"Local time": "time"}, inplace=True)
data.set_index("time", inplace=True)
#data.columns = [column.lower() for column in data.columns]
data.head()

,Open,High,Low,Close,Volume
time,,,,,
01.02.2021 00:00:00.000 GMT-0500,0.89187,0.89199,0.89155,0.89171,1055.5601
01.02.2021 01:00:00.000 GMT-0500,0.89172,0.89179,0.89136,0.89153,993.1500
01.02.2021 02:00:00.000 GMT-0500,0.89153,0.89222,0.89146,0.89187,2582.9299
01.02.2021 03:00:00.000 GMT-0500,0.89188,0.89512,0.89127,0.89511,8633.9502
01.02.2021 04:00:00.000 GMT-0500,0.89511,0.89570,0.89411,0.89479,6945.9502


In [4]:
data = data.query("Volume != 0")

In [5]:
import numpy as np
import pandas_ta as ta

# average true range
data["atr"] = data.ta.atr(length=20)
# relative strength index
data["rsi"] = data.ta.rsi()

ImportError: dlopen(/Users/arturopolancolozano/opt/anaconda3/lib/python3.9/site-packages/talib/_ta_lib.cpython-39-darwin.so, 0x0002): symbol not found in flat namespace '_TA_ACOS'

In [ ]:
data.ta.indicators()